Log in to facebook?



In [ ]:

var enterFacebook = () => {
    console.log('Facebook: Sign in required');
    var credentials = getCredentials('facebook.com');
    return client.click('input[name*="email"]')
    .keys(credentials.username)
    .pause(1000)
    .then(() => console.log('Facebook: Require password'))
    .click('input[name*="pass"]')
    .keys(credentials.password)
    .submitForm('[type="submit"]')
    .pause(2000)
    .isExisting('.cp-challenge-form')
    .then(is => {
        if(is) {
            throw new Error('captcha');
        }
    });
}

var loginFacebook = () => {
    return client
    .getUrl()
    .then(url => {
        var loggedIn = url.indexOf('facebook') > -1 && url.indexOf('login') == -1;
        return loggedIn
            ? client
                .isVisible('input[name*="email"]')
                .then(is => is ? enterFacebook() : client)
            : client.url('https://www.facebook.com/')
                .isVisible('input[name*="email"]')
                .then(is => is ? enterFacebook() : client)
    })
}
module.exports = loginFacebook;
loginFacebook;



List Facebook threads?



In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var scrollFacebookThreads = (threads) => {
    return client
    .execute(() => {
        document.getElementsByClassName(
            'scrollable')[0]
            .scrollTop += 10000;
    })
    .pause(2000)
    .then(() => getFacebookThreads(threads))
};

var getFacebookThreads = (threads) => {
    return client    
    .execute(() => {
        var iterator = document.evaluate(
            '//*[contains(@data-href, "messages")]/@data-href',
            document, null,
            XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
        var co = []; var m;
        while ((m = iterator.iterateNext()) && co.push(m));
        return co;
    })
    .then(els => {
        var newThreads = els.value.filter(e => threads.indexOf(e) === -1);
        threads = threads.concat(newThreads);
        return newThreads.length > 0
            ? scrollFacebookThreads(threads)
            : Promise.resolve(threads);
    })
};

var threads, results;
var listFacebookThreads = (threads) => {
    try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
    catch (e) { threads = []; }
    return client.getUrl()
    .then(url => url.indexOf('/messages/t') == -1
        ? client
              .click('[data-tooltip-content="Messages"]')
              .click('a[href*="/messages/t"]')
        : Promise.resolve([]))
    .pause(2000)
    .then(() => getFacebookThreads(threads || []))
    .then(r => threads = r)
    .then(url => url.indexOf('/messages/archived') == -1
        ? client.url('https://www.facebook.com/messages/archived')
              .pause(1000)
              .click('.scrollable a[href="#"]')
        : Promise.resolve([]))
    .pause(2000)
    .then(() => getFacebookThreads(threads || []))
    // list all message threads
    .then(r => {
        result = r;
        fs.writeFileSync(
        project + '/facebook-threads.json',
        JSON.stringify(r, null, 4));
        return result;
    })
};
module.exports = listFacebookThreads;
listFacebookThreads;



Scrape facebook profile?
    
    

In [ ]:
var importer = require('../Core');

// TODO: pass profile path in as parameter becomes readFacebookProfileInfo
var readFacebookProfileInfo = () => {
    var profile = {};
    // TODO: check for profile path
    return client.execute(() => {
        var people = document.evaluate(
            '//*[contains(@class, "scrollable")]//h4[contains(., "Profile")]/parent::*//a',
            document, null,
            XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        if(!people) {
            return '';
        }
        people.setAttribute('target', '_self');
        return people;
    })
    .then(r => r.value != '' 
        ? client.elementIdClick(r.value.ELEMENT)
            .pause(2000)
            .then(() => importer.runAllPromises([
                client.getText('[role="main"] h1'),
                client.isExisting('#intro_container_id li').then(is => is 
                    ? client.getText('#intro_container_id li') 
                    : Promise.resolve('')),
                client.getUrl(),
            ]))
            .then(r => (profile = ({
                name: r[0],
                description: r[1],
                url: r[2],
            })))
        : client)
    .then(() => profile)
    .catch(e => console.log(e))
};
module.exports = readFacebookProfileInfo;
readFacebookProfileInfo;



Get messages from facebook?



In [ ]:
var importer = require('../Core');
var moment = require('moment');
var chrono = require('chrono-node');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var readFacebookProfileInfo;
var switchToParticipantThread = (i) => {
    return client
    .pause(1000)
    .click('//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li[' + i + ']')
    .pause(1000)
    .then(() => readFacebookProfileInfo())
    .catch(e => console.log(e))
};

var getThreadParticipants = (thread) => {
    return client.then(() => importer.interpretAll(['scrape facebook profile']))
    .then(r => r[0].runFresherInNewContext({client}))
    .then(r => readFacebookProfileInfo = r)
    .execute(() => {
        var people = document.evaluate(
            'count(//*[contains(@class, "scrollable")]//h4[contains(., "People")]/parent::*//li)',
            document, null,
            XPathResult.NUMBER_TYPE, null).numberValue;
        return people;
    })
    .then(r => {
        var result = [];
        for(var i = 1; i < Math.min(3, r.value); i++) {
            result[result.length] = (i =>
                resolve => {
                var profile = {};
                switchToParticipantThread(i)
                .then(r => profile = r)
                .url(thread)
                .pause(3000)
                .catch(e => console.log(e))
                .then(() => resolve(profile))
            })(i);
        }
        if(r.value === 0) {
            result[result.length] = (resolve => {
                var profile = {};
                readFacebookProfileInfo()
                .then(r => profile = r)
                .url(thread)
                .pause(3000)
                .then(() => resolve(profile))
                .catch(e => console.log(e))
            })
        }
        return importer.runAllPromises(result).catch(e => console.log(e));
    })
    .catch(e => console.log(e))
};

var readFacebookMessages = (messages, i = 0) => {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client
    .execute(() => {
        var result = [];
        var groups = document.getElementsByClassName('scrollable')[2];
        var times = groups.getElementsByTagName('h4');
        for(var t = 0; t < times.length; t++) {
            var bubble = times[t];
            while((bubble = bubble.nextSibling) && bubble.nodeName == 'DIV') {
                var from = bubble.getElementsByTagName('h5')[0];
                result[result.length] = {
                    from: typeof from !== 'undefined' ? from.textContent : '',
                    time: times[t].innerText,
                    message: bubble.innerText
                };
            }
        }
        return result;
    })
    .then(r => {
        var newMessages = r.value
            .filter(e => messages
                .filter(m => m.message == e.message).length === 0);
        messages = newMessages.concat(messages);
        return newMessages.length > 0
                ? scrollClient('//*[contains(@role, "main")]//*[contains(@class, "scrollable")][1]', true, i)
                    .pause(2000)
                    .then(() => i > 3 
                          ? messages 
                          : readFacebookMessages(messages, i + 1))
                : Promise.resolve(messages);
    })
    .catch(e => console.log(e))
};

var readFacebookThread = (thread, messages) => {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop()
        .replace(/[^a-z0-9]/ig, '_');
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    try { messages = JSON.parse(fs.readFileSync(file)) }
    catch (e) { messages = [] }
    var participants;
    return client.getUrl().then(url => url.indexOf(thread) == -1
        ? client.url(thread)
        : client)
    // get participants from topcard
    .pause(1000)
    .then(() => getThreadParticipants(thread))
    .then(p => (participants = p))
    
    // TODO: save to contacts
    .then(() => readFacebookMessages(messages || []))
    .then(messages => ({
        thread: thread,
        participants: participants || [],
        messages: messages.map(m => {
            var newTime = chrono.parseDate(m.time);
            newTime.setHours(newTime.getHours()-(new Date()).getTimezoneOffset()/60)
            return Object.assign(m, {time: newTime})
        })
    }))
    .then(t => {
        var filename = project + '/' 
            + threadId
            + '-' + t.participants
            .map(p => p.name).join('')
            .replace(/[^a-z0-9]/ig, '_') + '.json';
        fs.writeFileSync(filename, JSON.stringify(t, null, 4));
        return t;
    })
    .catch(e => console.log(e))
};
module.exports = readFacebookThread;
readFacebookThread;



Like all facebook posts?



In [ ]:
var glob = require('glob');
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var listFacebookPosts = (profile, posts) => {
    var participants;
    return client
    .then(() => getAllUntil(
        '//body',
        '//*[contains(@class, "fbUserPost")]//*[contains(@class, "timestampContent")]/parent::*/parent::*/@href',
        posts,
        (a, b) => a == b,
        (i) => i < 50
    ))
    // transform add facebook url
    .then(r => r.map(l => l.indexOf('facebook.com') == -1
        ? ('https://www.facebook.com' + l)
        : l))
    .catch(e => console.log(e))
};

var scrapeFacebookPost = (post) => {
    return client
    .getUrl().then(url => url.indexOf(post) == -1
        ? client.url(post)
        : client)
    .pause(3000)
    .then(() => getAllXPath([
'//body[not(.//*[contains(@class, "fbPhotoSnowliftPopup")])]|//body[.//*[contains(@class, "fbPhotoSnowliftPopup")]]//*[contains(@class, "fbPhotoSnowliftPopup")]',
        {
            posts: [
'.//*[contains(@class, "fbPhotoSnowliftAuthorInfo")]|.//*[contains(@class, "fbUserPost")]',
                {
                    description: 
'.//following-sibling::div//*[contains(@class, "fbPhotosPhotoCaption")]//text()|.//*[contains(@class, "userContent")]//text()|.//h5//text()|.//a[contains(@class, "profileLink")]//text()',
                    participants: [
'.//a[contains(@class, "profileLink")]/@href|.//a[contains(@href, "facebook") and .//img]/@href',
'./following-sibling::div//a/@href',
'.//*[contains(@class, "commentable_item")]//a[contains(@class, "UFICommentActorName")]/@href',
                    ],
                    comments: [
'.//h6[contains(., "Comments")]//following-sibling::div/div/div[contains(@class, "UFIComment")]',
                        {
                            time: 
'.//*[contains(@class, "uiLinkSubtle")]//text()',
                            content: 
'.//*[contains(@class, "UFICommentBody")]//text()',
                            from: 
'.//a[contains(@class, "UFICommentActorName")]/text()|.//a[contains(@class, "UFICommentActorName")]/@href'
                        }
                    ]
                }
            ]
        }
    ]))
    .then(r => r[0].posts.map(p => {
        return Object.assign(p, {
            post: post,
            description: p.description.join(' '),
            participants: p.participants.filter(i => i
                .indexOf('photo') == -1 
                && i != '#'
                && i.indexOf('ufi/reaction') == -1),
            photos: p.participants.filter(i => i
                .indexOf('photo') != -1
                && i != '#'
                && i.indexOf('ajax/sharer') == -1),
            comments: p.comments.map(c => Object.assign(c, {
                time: c.time.join(' '),
                content: c.content.join(' '),
                from: c.from.join(' '),
            }))
        });
    }))
    .catch(e => console.log(e))
};

var likeFacebookPost = (c) => {
    console.log(c);
    var result;
    return (resolve) => scrapeFacebookPost(c)
    .then(r => result = r)
    .elements('//body[not(.//*[contains(@class, "fbPhotoSnowliftPopup")])]//a[contains(., "Like")][contains(@class, "UFILikeLink")][not(contains(@class, "UFILinkBright"))]|//body[.//*[contains(@class, "fbPhotoSnowliftPopup")]]//*[contains(@class, "fbPhotoSnowliftPopup")]//a[contains(., "Like")][contains(@class, "UFILikeLink")][not(contains(@class, "UFILinkBright"))]')
    .then(els => {
        var result = importer.runAllPromises(els.value.slice(0,4)
        .map(el => (resolve) => client.elementIdClick(el.ELEMENT)
            .pause(1500)
            .then(() => resolve())
            .catch(e => resolve())))
        .catch(e => console.log(e))
        return result;
    })
    .then(() => resolve(result))
    .catch(e => console.log(e));
}

var posts, getAllXPath, getAllUntil;
var likeAllPosts = (profile, like = null) => {
    var profileId = profile.replace(/^\/|\/$/ig, '').split('/').pop()
        .replace(/[^a-z0-9]/ig, '_');
    var file = glob.sync('**/' + profileId + '-*.json', {cwd: project})[0];
    try { posts = JSON.parse(fs.readFileSync(file)) }
    catch (e) { posts = [] }
    return client
    .then(() => importer.interpretAll([
        'get all elements until',
        'get all elements',
    ]))
    .then(r => {
        getAllUntil = r[0].runFresherInNewContext({client});
        getAllXPath = r[1].runFresherInNewContext({client});
        return client.getUrl();
    })
    .then(url => url != profile
        ? client.url(profile)
        : client)
    .then(() => listFacebookPosts(posts.map(p => p.post)))
    // TODO: remove slice to download all posts from first part
    .then(r => importer.runAllPromises(r.map(c => likeFacebookPost(c))))
    .then(t => {
        var filename = project + '/'
            + profileId
            + '-posts-' + (new Date()).getFullYear() + '.json';
        fs.writeFileSync(filename, JSON.stringify(t, null, 4));
        return t;
    })
    .catch(e => console.log(e))
};
module.exports = likeAllPosts;
likeAllPosts;



In [ ]:
var importer = require('../Core');
var fs = require('fs');

$$.async();
var listFacebookThreads, readFacebookThread, runSeleniumCell;
importer.interpretAll(['selenium cell'])
.then(r => {
    runSeleniumCell = r[0].runInNewContext();
    return runSeleniumCell([
        'log in facebook',
        'list facebook threads',
        'messages from facebook',
        'like all facebook posts',
    ]);
})
.then(r => {
    loginFacebook = r[0];
    listFacebookThreads = r[1];
    readFacebookThread = r[2];
    likeAllPosts = r[3];
    return loginFacebook();
})
//.then(() => listFacebookThreads())
//.then(threads => {
//    try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
//    catch (e) { threads = []; }
//    return importer.runAllPromises(threads.slice(450, 600)
//        .map(t => ((resolve) => readFacebookThread(t).then((r) => resolve(r)))));
//})
.then(r => importer.runAllPromises([
    'https://www.facebook.com/talbert.tso'
].map(p => (resolve) => likeAllPosts(p).then(r => resolve(r)))))

/*
https://www.facebook.com/johan.larson.56
https://www.facebook.com/dadsrawesome/videos/1165913990203850/
https://www.facebook.com/galacticempireofficial/videos/800461353413445/
    */
    
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e))



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import n

Scrape facebook events?



In [ ]:
var importer = require('../Core');
var getAllXPath, getAllUntil;

var scrapeFacebookEvent = (event) => {
    console.log(event);
    var description;
    return client
    .then(() => importer.interpretAll([
        'get all elements until',
        'get all elements',
    ]))
    .then(r => {
        getAllUntil = r[0].runFresherInNewContext({client});
        getAllXPath = r[1].runFresherInNewContext({client});
        return client.getUrl();
    })
    .then(url => url.indexOf(event) == -1
        ? client.url(event)
        : client)
    .pause(3000)
    .isExisting('//a[contains(., "About")]')
    .then(is => is ? client.click('//a[contains(., "About")]') : client)
    .pause(1000)
    .then(() => getAllXPath([
'//body[not(.//*[contains(@class, "fbPhotoSnowliftPopup")])]|//body[.//*[contains(@class, "fbPhotoSnowliftPopup")]]//*[contains(@class, "fbPhotoSnowliftPopup")]',
        {
            description: [
                './/*[contains(@id, "reaction_units")]/div/div|.//*[contains(@id, "event_summary")]//li',
                {
                    value: ['.//span//text()']
                }
            ]
        }
    ]))
    .then(desc => {
        description = desc;
    })
    .isExisting('//a[contains(., "Discussion")]')
    .then(is => is ? client.click('//a[contains(., "Discussion")]') : client)
    .pause(1000)
    .then(() => {
        return getAllXPath([
'//body[not(.//*[contains(@class, "fbPhotoSnowliftPopup")])]|//body[.//*[contains(@class, "fbPhotoSnowliftPopup")]]//*[contains(@class, "fbPhotoSnowliftPopup")]',
        {
            posts: [
'.//*[contains(@class, "fbPhotoSnowliftAuthorInfo")]|.//*[contains(@class, "fbUserPost")]',
                {
                    description: 
'.//following-sibling::div//*[contains(@class, "fbPhotosPhotoCaption")]//text()|.//*[contains(@class, "userContent")]//text()|.//h5//text()|.//a[contains(@class, "profileLink")]//text()',
                    participants: [
'.//a[contains(@class, "profileLink")]/@href|.//a[contains(@href, "facebook") and .//img]/@href',
'./following-sibling::div//a/@href',
'.//*[contains(@class, "commentable_item")]//a[contains(@class, "UFICommentActorName")]/@href',
                    ],
                    comments: [
'.//h6[contains(., "Comments")]//following-sibling::div/div/div[contains(@class, "UFIComment")]',
                        {
                            time: 
'.//*[contains(@class, "uiLinkSubtle")]//text()',
                            content: 
'.//*[contains(@class, "UFICommentBody")]//text()',
                            from: 
'.//a[contains(@class, "UFICommentActorName")]/text()|.//a[contains(@class, "UFICommentActorName")]/@href'
                        }
                    ]
                }
            ]
        }
        ])
    })
    .then(results => results.map((r, i) => Object.assign(r, {
        description: description[i].description.map(d => d.value.join(' ')),
        posts: r.posts.map(p => {
            return Object.assign(p, {
                event: event,
                description: p.description.join(' '),
                participants: p.participants.filter(i => i
                    .indexOf('photo') == -1 
                    && i != '#'
                    && i.indexOf('ufi/reaction') == -1),
                photos: p.participants.filter(i => i
                    .indexOf('photo') != -1
                    && i != '#'
                    && i.indexOf('ajax/sharer') == -1),
                comments: p.comments.map(c => Object.assign(c, {
                    time: c.time.join(' '),
                    content: c.content.join(' '),
                    from: c.from.join(' '),
                }))
            });
        })
    })))
    .catch(e => console.log(e))
};
module.exports = scrapeFacebookEvent;
scrapeFacebookEvent;



Scrape a bunch of facebook events from some html input?



In [ ]:
var glob = require('glob');
var fs = require('fs');
var path = require('path');
var importer = require('../Core')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';
var filename = path.join(project, 'events.json');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
};
var getEvents = (file) => {
    var html = fs.readFileSync(file).toString();
    return regexToArray(/href="([^"?]*)/ig, html, 1)
    .filter(h => h.indexOf('.js') == -1)
    .filter(h => h.indexOf('.css') == -1)
    .filter(h => h.match(/events\/[0-9]+/ig));
};

var eventFiles = glob.sync('**/*@(Past|Events|Cullinan).htm',
                          {cwd: project});
$$.async();
var scrapeFacebookEvent;
importer.interpretAll(['selenium cell'])
.then(r => {
    runSeleniumCell = r[0].runInNewContext();
    return runSeleniumCell([
        'log in facebook',
        'scrape facebook events'
    ]);
})
.then(r => {
    loginFacebook = r[0];
    scrapeFacebookEvent = r[1];
    return loginFacebook();
})
.then(() => {
    return importer.runAllPromises([
    ...getEvents(path.join(project, eventFiles[0])),
    ...getEvents(path.join(project, eventFiles[1])),
    ...getEvents(path.join(project, eventFiles[2]))
    ]
    .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
    .map(e => resolve => scrapeFacebookEvent(e)
         .then(r => resolve(r))
         .catch(r => resolve(r))))
})
.then(r => {
    fs.writeFileSync(filename, JSON.stringify(r, null, 4));
    $$.sendResult(r);
})
.catch(e => $$.sendError(e))



Automatically accept friend requests?



In [ ]:
var glob = require('glob');
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
};
var getFriends = (file) => {
    var html = fs.readFileSync(file).toString();
    return regexToArray(/href="([^"]*)/ig, html, 1)
    .filter(h => h.indexOf('.js') == -1 && h.indexOf('.css') == -1
            && h.indexOf('friends_tab') > -1)
    .map(m => (/(.*)\?/ig).exec(m)[1])
    .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
};

var friends = glob.sync('**/*friend*', {cwd: project, nodir: true});
console.log(friends);
var urls = getFriends(path.join(project, friends[0]));
var urls2 = getFriends(path.join(project, friends[1]));
var diff = urls.filter(u => urls2.indexOf(u) == -1).concat(urls2.filter(u => urls.indexOf(u) == -1))
$$.sendResult(diff);



In [ ]:
// How many facebook friends are also on linked in?  Do people still post contact information?


